# Basics of Dash

## Installation

We are going to use Google Colab to deploy in a localhost server a Dash application. In this way we will avoid using environments or further configuration steps in our computers. We only need to install dash by means of the python package manager pip.

In [1]:
!pip install dash

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install jupyter-dash

Defaulting to user installation because normal site-packages is not writeable


## Libraries

Once we installed them we import `pandas` in order to deal with the dataset as usual. `plotly` to make the visualizations. And finally `Dash` to build an small static website.

In [3]:
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px

from dash import dcc
from dash import html
from jupyter_dash import JupyterDash

## Reading the dataset

In [4]:
url = 'https://www.famaf.unc.edu.ar/~nocampo043/sysarmy_survey_2020_processed.csv'
df = pd.read_csv(url)

## What do we plot?

After the exploration process we need to choose the plots we are going to show to the our clients, supervisors, boss. In this case we are going to come back to previous plots of the nobetook 04. Our choices plots related to the `salary_monthly_NET` variable and the `profile_studies_level`. In this case 3 types of barplots.

We saved each of these plots as figures objects (note the fig suffix we added to their name).

In [5]:
sorted_studies_levels = ["Primary", "Secondary", "Terciary", "University",
                         "Postgraduate", "Doctorate", "Postdoc"]

# Barplot of the profile_studies_level alongside their frequency count.
df_studies_level_count = (
    df["profile_studies_level"]
      .value_counts()
      .reset_index()
      .rename(columns={"index": "profile_studies_level",
                       "profile_studies_level": "count"})
)

studies_level_count_fig = px.bar(df_studies_level_count,
                                 x='profile_studies_level',
                                 y='count',
                                 category_orders=dict(profile_studies_level=sorted_studies_levels))

# Barplot of the profile_studies_level alongside the mean of the salary

df_studies_level_mean = (
    df[["profile_studies_level", "salary_monthly_NET"]]
      .groupby("profile_studies_level")
      .mean()
      .reset_index()
      .rename(columns={"salary_monthly_NET": "salary_monthly_NET_mean"})
)

studies_level_mean_fig = px.bar(df_studies_level_mean,
                                x='profile_studies_level',
                                y='salary_monthly_NET_mean',
                                category_orders=dict(profile_studies_level=sorted_studies_levels))


# Barplot using grouped studies level alongside the mean of the salary
new_groups = {
    'Postdoc': 'Postgraduate',
    'Doctorate': 'Postgraduate',
    'Primary': 'Pre-degree',
    'Secondary': 'Pre-degree'
}
order = ['Pre-degree', 'Terciary', 'University', 'Postgraduate']
grouped_studies_level = df.profile_studies_level.replace(new_groups)
df["grouped_profile_studies_level"] = grouped_studies_level

df_grouped_studies_level_mean = (
    df[["grouped_profile_studies_level", "profile_studies_level_state", "salary_monthly_NET"]]
      .groupby(["grouped_profile_studies_level", "profile_studies_level_state"])
      .mean()
      .reset_index()
      .rename(columns={"salary_monthly_NET": "salary_monthly_NET_mean"})
)

grouped_studies_level_fig = px.bar(df_grouped_studies_level_mean,
                                   x='grouped_profile_studies_level',
                                   y='salary_monthly_NET_mean',
                                   color='profile_studies_level_state',
                                   barmode='group',
                                   category_orders=dict(grouped_profile_studies_level=order))

## HTML and CSS

HTML is a declarative language to specify the layout of a webpage.

Cascading Style Sheets (CSS) is used to format the layout of a webpage.

With CSS, you can control the color, font, the size of text, the spacing between elements, how elements are positioned and laid out, what background images or background colors are to be used, different displays for different devices and screen sizes, and much more!

In this code cell we formated each section in our layout and set certain visualization properties. For example the header has a dark blue color with a height of 256 pixeles.

In [6]:
header = {
    "background-color": "#222222",
    "height": "256px",
    "display": "flex",
    "flex-direction": "column",
    "justify-content": "center;",
}

header_title = {
    'color': '#FFFFFF',
    'font-size': '48px',
    'font-weight': 'bold',
    'text-align': 'center',
    'margin': '0 auto',
}

header_description = {
    'color': '#CFCFCF',
    'margin': '4px auto',
    'text-align': 'center',
    'max-width': '384px',
}

plot_card = {
    'margin-bottom': '24px',
    'box-shadow': '0 4px 6px 0 rgba(0, 0, 0, 0.18)'
}

wrapper = {
    'margin-right': 'auto',
    'margin-left': 'auto',
    'max-width': '1024px',
    'padding-right': '10px',
    'padding-left': '10px',
    'margin-top': '32px',
    'font-family': '"Lato", sans-serif',
    'background-color': '#F7F7F7'
}

Once we defined the styles and the layout we can combine them using slash. Here Div objects correspond to `<div>` tags in html, the parameter `children` indicates all the following tags that depend on the `div` object. Once our layout is finished we can run our application server in external mode so it can be run in your localhost in this case in http://127.0.0.1:8050/ .

In [7]:
app = JupyterDash(__name__)
app.layout = html.Div(
    children=[
        html.Div(
            children=[
                html.H1(children="Using Dash", style=header_title),
                html.P(
                    children="Analyze the salary monthly net alongside the years of experience",
                    style=header_description
                )     
            ],
            style=header),
        html.Div(
            children=[
                dcc.Graph(figure=studies_level_count_fig),
                dcc.Graph(figure=studies_level_mean_fig),
                dcc.Graph(figure=grouped_studies_level_fig)         
            ],
            style=plot_card,
        )
    ],
    style=wrapper
)

if __name__ =='__main__':
    app.run_server(mode="external")

Dash app running on http://127.0.0.1:8050/
